## Phase 1
#### ETL - Extract, Transform, and Load data into Delta Table

In [0]:
# 1. Create new database
spark.sql("CREATE DATABASE IF NOT EXISTS usa_housing")

DataFrame[]

In [0]:
# 2. Create table 
df = spark.table("usa_housing.realtor_data")

In [0]:
# 3. Display created df
display(df.limit(10))

brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
25793.0,sold,332900.0,5,3,null,564331.0,Pensacola,Florida,32534,2361.0,2022-03-31
25793.0,sold,318900.0,5,3,0.16,582075.0,Pensacola,Florida,32534,2361.0,2022-03-31
25793.0,sold,327900.0,5,3,0.16,566668.0,Pensacola,Florida,32534,2361.0,2022-03-31
25793.0,sold,288900.0,3,2,null,580468.0,Pensacola,Florida,32534,1611.0,2022-03-31
25793.0,sold,302900.0,3,2,null,568824.0,Pensacola,Florida,32534,1611.0,2022-03-31
25793.0,sold,300900.0,3,2,null,571417.0,Pensacola,Florida,32534,1611.0,2022-03-31
15376.0,sold,99000.0,null,null,9.34,440848.0,Cantonment,Florida,32533,null,2021-12-10
11267.0,sold,354900.0,3,2,2.02,713767.0,Cantonment,Florida,32533,2950.0,2021-12-27
55622.0,sold,419900.0,4,3,0.34,1156905.0,Pensacola,Florida,32533,2744.0,2021-12-27
53231.0,sold,215000.0,4,3,0.23,837543.0,Cantonment,Florida,32533,2078.0,2021-12-27


In [0]:
# 4. Print schema
df.printSchema()

root
 |-- brokered_by: double (nullable = true)
 |-- status: string (nullable = true)
 |-- price: double (nullable = true)
 |-- bed: long (nullable = true)
 |-- bath: long (nullable = true)
 |-- acre_lot: double (nullable = true)
 |-- street: double (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip_code: long (nullable = true)
 |-- house_size: double (nullable = true)
 |-- prev_sold_date: date (nullable = true)



In [0]:
# 5. Count NULLs in each column
from pyspark.sql.functions import col, when, count

df.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df.columns
]).show()

+-----------+------+-----+------+------+--------+------+----+-----+--------+----------+--------------+
|brokered_by|status|price|   bed|  bath|acre_lot|street|city|state|zip_code|house_size|prev_sold_date|
+-----------+------+-----+------+------+--------+------+----+-----+--------+----------+--------------+
|       4533|     0| 1541|481317|511771|  325589| 10866|1407|    8|     299|    568484|        734297|
+-----------+------+-----+------+------+--------+------+----+-----+--------+----------+--------------+



In [0]:
# 6. Drop useless columns like status and prev_sold_date
df_new = df.drop("status", "prev_sold_date")

In [0]:
# 7. Display new df
display(df_new.limit(10))

brokered_by,price,bed,bath,acre_lot,street,city,state,zip_code,house_size
25793.0,332900.0,5,3,null,564331.0,Pensacola,Florida,32534,2361.0
25793.0,318900.0,5,3,0.16,582075.0,Pensacola,Florida,32534,2361.0
25793.0,327900.0,5,3,0.16,566668.0,Pensacola,Florida,32534,2361.0
25793.0,288900.0,3,2,null,580468.0,Pensacola,Florida,32534,1611.0
25793.0,302900.0,3,2,null,568824.0,Pensacola,Florida,32534,1611.0
25793.0,300900.0,3,2,null,571417.0,Pensacola,Florida,32534,1611.0
15376.0,99000.0,null,null,9.34,440848.0,Cantonment,Florida,32533,null
11267.0,354900.0,3,2,2.02,713767.0,Cantonment,Florida,32533,2950.0
55622.0,419900.0,4,3,0.34,1156905.0,Pensacola,Florida,32533,2744.0
53231.0,215000.0,4,3,0.23,837543.0,Cantonment,Florida,32533,2078.0


In [0]:
# 8. Fill missing values with standard values
df_final = df_new.na.fill({
    "brokered_by": 0,
    "price": 0.0,
    "bed": 0,
    "bath": 0,
    "acre_lot": 0.0,
    "street": -1.0, # fake street
    "city": "unknown",
    "state": "unknown",
    "zip_code": -1.0, # fake zip-code
    "house_size": 0.0
})


In [0]:
# 9. Check for nulls again
df_final.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df_final.columns
]).show()

+-----------+-----+---+----+--------+------+----+-----+--------+----------+
|brokered_by|price|bed|bath|acre_lot|street|city|state|zip_code|house_size|
+-----------+-----+---+----+--------+------+----+-----+--------+----------+
|          0|    0|  0|   0|       0|     0|   0|    0|       0|         0|
+-----------+-----+---+----+--------+------+----+-----+--------+----------+



In [0]:
# 10. Display final df
display(df_final.limit(10))

brokered_by,price,bed,bath,acre_lot,street,city,state,zip_code,house_size
25793.0,332900.0,5,3,0.0,564331.0,Pensacola,Florida,32534,2361.0
25793.0,318900.0,5,3,0.16,582075.0,Pensacola,Florida,32534,2361.0
25793.0,327900.0,5,3,0.16,566668.0,Pensacola,Florida,32534,2361.0
25793.0,288900.0,3,2,0.0,580468.0,Pensacola,Florida,32534,1611.0
25793.0,302900.0,3,2,0.0,568824.0,Pensacola,Florida,32534,1611.0
25793.0,300900.0,3,2,0.0,571417.0,Pensacola,Florida,32534,1611.0
15376.0,99000.0,0,0,9.34,440848.0,Cantonment,Florida,32533,0.0
11267.0,354900.0,3,2,2.02,713767.0,Cantonment,Florida,32533,2950.0
55622.0,419900.0,4,3,0.34,1156905.0,Pensacola,Florida,32533,2744.0
53231.0,215000.0,4,3,0.23,837543.0,Cantonment,Florida,32533,2078.0


In [0]:
# 11. Save new data as delta table
df_final.write.mode("overwrite").format("delta").saveAsTable("usa_housing.clean2_data")


In [0]:
# 12. Print new schema
df_final.printSchema()

root
 |-- brokered_by: double (nullable = false)
 |-- price: double (nullable = false)
 |-- bed: long (nullable = false)
 |-- bath: long (nullable = false)
 |-- acre_lot: double (nullable = false)
 |-- street: double (nullable = false)
 |-- city: string (nullable = false)
 |-- state: string (nullable = false)
 |-- zip_code: long (nullable = true)
 |-- house_size: double (nullable = false)



## Phase 2
#### EDA - Exploratory Data Analysis

In [0]:
display(spark.sql("""
-- 1. Display table
SELECT * 
FROM usa_housing.clean2_data
LIMIT 10;"""))

brokered_by,price,bed,bath,acre_lot,street,city,state,zip_code,house_size
103378.0,105000.0,3,2,0.12,1962661.0,Adjuntas,Puerto Rico,601,920.0
52707.0,80000.0,4,2,0.08,1902874.0,Adjuntas,Puerto Rico,601,1527.0
103379.0,67000.0,2,1,0.15,1404990.0,Juana Diaz,Puerto Rico,795,748.0
31239.0,145000.0,4,2,0.1,1947675.0,Ponce,Puerto Rico,731,1800.0
34632.0,65000.0,6,2,0.05,331151.0,Mayaguez,Puerto Rico,680,0.0
103378.0,179000.0,4,3,0.46,1850806.0,San Sebastian,Puerto Rico,612,2520.0
1205.0,50000.0,3,1,0.2,1298094.0,Ciales,Puerto Rico,639,2040.0
50739.0,71600.0,3,2,0.08,1048466.0,Ponce,Puerto Rico,731,1050.0
81909.0,100000.0,2,1,0.09,734904.0,Ponce,Puerto Rico,730,1092.0
65672.0,300000.0,5,3,7.46,1946226.0,Las Marias,Puerto Rico,670,5403.0


In [0]:
display(spark.sql("""
-- 2. Top 10 states by number of listings
SELECT state, COUNT(*) AS listing_count
FROM usa_housing.clean2_data
GROUP BY state
ORDER BY listing_count DESC
LIMIT 10;"""))

state,listing_count
Florida,249432
California,227215
Texas,208335
New York,103159
North Carolina,85745
Illinois,85280
Georgia,80977
Pennsylvania,78373
Arizona,72825
Virginia,68763


In [0]:
display(spark.sql("""
-- 3. Top 10 Cities with most listings
SELECT city, COUNT(*) AS listing_count
FROM usa_housing.clean2_data
GROUP BY city
ORDER BY listing_count DESC
LIMIT 10;"""))

city,listing_count
Houston,23862
Chicago,18238
New York City,12634
Jacksonville,11743
Philadelphia,10449
Miami,9737
Los Angeles,8984
Tucson,8943
Dallas,8569
Richmond,8360


In [0]:
display(spark.sql("""
-- 4. Average price of houses by state
SELECT state, ROUND(AVG(price), 2) AS avg_price
FROM usa_housing.clean2_data
GROUP BY state
ORDER BY avg_price DESC;"""))

state,avg_price
Hawaii,1240095.3
California,1029074.22
New York,943778.53
District of Columbia,882849.78
Utah,881297.3
Colorado,831818.5
Montana,792043.24
Virgin Islands,759440.05
Massachusetts,724824.71
Guam,713861.66


In [0]:
display(spark.sql("""
-- 5. House size bins
SELECT FLOOR(house_size / 500) * 500 AS size_bin,
       COUNT(*) AS count
FROM usa_housing.clean_data
WHERE house_size > 0
GROUP BY size_bin
ORDER BY size_bin
"""))

size_bin,count
0,7733
500,154335
1000,431678
1500,415413
2000,269792
2500,154918
3000,90857
3500,48477
4000,28874
4500,16694


In [0]:
display(spark.sql("""
-- 6. Top 3 most expensive listings by state
SELECT state, city, price
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY state ORDER BY price DESC) AS rank
    FROM usa_housing.clean2_data
)
WHERE rank <= 3
"""))


state,city,price
Alabama,Bay Minette,2.63055E7
Alabama,Gallion,1.9999E7
Alabama,Salem,1.9E7
Alaska,Homer,9000000.0
Alaska,Palmer,8950000.0
Alaska,Anchorage,6674025.0
Arizona,Paradise Valley,3.06E7
Arizona,Scottsdale,2.95E7
Arizona,Congress,2.5E7
Arkansas,Bentonville,1.878875E7


In [0]:
display(spark.sql("""
-- 7. Top 50 most expensive states by price per sqft
SELECT *,
       CASE WHEN house_size > 0 THEN price / house_size ELSE NULL END AS price_per_sqft
FROM usa_housing.clean2_data
ORDER BY price_per_sqft DESC
LIMIT 50
"""))


brokered_by,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,price_per_sqft
23826.0,2.1474836E9,2,2,0.12,11355.0,International,California,-1,885.0,2426535.1412429377
13872.0,5.15E8,2,2,2.59,1658312.0,San Diego,California,92119,1048.0,491412.213740458
98430.0,1850000.0,4,4,0.0,699459.0,Laurel,New York,11948,4.0,462500.0
100472.0,8.75E8,9,2,0.0,1811391.0,Bronx,New York,10458,2440.0,358606.5573770492
43753.0,9.5E7,0,0,120.0,362431.0,Beverly Hills,California,90210,741.0,128205.1282051282
41839.0,5.0E7,1,1,650.0,169262.0,Snowmass,Colorado,81654,1092.0,45787.54578754579
79515.0,5.5E7,3,2,0.0,1002957.0,Clifton,Colorado,81520,1248.0,44070.51282051282
16829.0,1.09E8,3,4,4.21,904983.0,Carpinteria,California,93013,2514.0,43357.19968178202
97348.0,2954050.0,0,0,118.16,3720.0,Bertram,Texas,78605,100.0,29540.5
23592.0,1.0E8,5,5,8.0,484175.0,Atherton,California,94027,3731.0,26802.46582685607


In [0]:
display(spark.sql("""
-- 8. Top 10 cities with most sales
SELECT city, state, COUNT(*) AS listings, SUM(price) AS price
FROM usa_housing.clean2_data
GROUP BY city, state
ORDER BY price DESC
LIMIT 10
"""))

city,state,listings,price
New York City,New York,12634,2.8628921743E10
Los Angeles,California,8984,1.7224385241E10
New York,New York,5353,1.5843133188E10
Houston,Texas,23603,1.1220531504E10
Chicago,Illinois,18238,8.975090754E9
Miami,Florida,9616,8.286651959E9
San Francisco,California,4605,8.155128354E9
San Diego,California,7059,7.70378736E9
Naples,Florida,5910,7.230144398E9
Brooklyn,New York,5750,6.603006472E9


In [0]:
display(spark.sql("""
-- 9. Top 10 cities with least sales
SELECT city, state, COUNT(*) AS listings, SUM(price) AS price
FROM usa_housing.clean2_data
WHERE price > 0
GROUP BY city, state
ORDER BY price ASC
LIMIT 10
"""))

city,state,listings,price
North 27th Saint,Nebraska,1,1.0
Holy Cross,Iowa,1,1.0
Bernard,Iowa,1,1.0
Curoe Rd,Iowa,1,1.0
Greenville,Iowa,1,1.0
Wadena,Iowa,1,1.0
King Ave,Iowa,1,1.0
L Ave,Iowa,1,1.0
Sparkman,Arkansas,1,1.0
Killbuck,Ohio,1,1.0


In [0]:
display(spark.sql("""
-- 10. Top 10 zip codes with largest lots
SELECT zip_code, AVG(acre_lot) AS avg_lot_size
FROM usa_housing.clean2_data
GROUP BY zip_code
ORDER BY avg_lot_size DESC
LIMIT 10
"""))


zip_code,avg_lot_size
81029,23413.75
69212,15300.0
6353,13762.785
15436,13025.724285714286
59713,11100.0
59520,11002.62
59077,9296.14
12022,8347.449166666667
97711,7176.344
12838,6673.593333333333


In [0]:
display(spark.sql("""
-- 11. Top 10 zip codes with smallest lots
SELECT zip_code, AVG(acre_lot) AS avg_lot_size
FROM usa_housing.clean2_data
WHERE acre_lot > 0
GROUP BY zip_code
ORDER BY avg_lot_size ASC
LIMIT 10
"""))


zip_code,avg_lot_size
89595,0.01
12509,0.01
33867,0.01
8444,0.01
8862,0.01
10975,0.01
95113,0.015
43217,0.015
93634,0.0175
37201,0.01875


In [0]:
display(spark.sql("""
-- 12. Top 10 brokers with most listings
SELECT brokered_by, COUNT(*) AS num_listings
FROM usa_housing.clean2_data
GROUP BY brokered_by
ORDER BY num_listings DESC
LIMIT 10
"""))


brokered_by,num_listings
22611.0,45658
16829.0,27732
53016.0,21709
23592.0,9176
30807.0,8464
33714.0,6928
57595.0,6410
84534.0,5502
109978.0,5365
109914.0,5231


In [0]:
display(spark.sql("""
-- 13. Top 10 brokers with least listings
SELECT brokered_by, COUNT(*) AS num_listings
FROM usa_housing.clean2_data
GROUP BY brokered_by
ORDER BY num_listings ASC
LIMIT 10
"""))


brokered_by,num_listings
80661.0,1
98818.0,1
77629.0,1
38269.0,1
37986.0,1
7150.0,1
99178.0,1
12408.0,1
8311.0,1
88463.0,1


In [0]:
display(spark.sql("""
SELECT *
FROM usa_housing.clean2_data
WHERE acre_lot > 10000                  
"""))

brokered_by,price,bed,bath,acre_lot,street,city,state,zip_code,house_size
88715.0,585000.0,4,4,100000.0,1930360.0,San Juan,Puerto Rico,926,3300.0
10554.0,150000.0,0,0,99999.0,595393.0,Simsbury,Connecticut,6070,0.0
10560.0,5700000.0,0,0,99999.0,1106245.0,Middletown,Connecticut,6457,99999.0
76006.0,250000.0,3,3,18817.0,1670131.0,Narragansett,Rhode Island,2882,1532.0
42180.0,319000.0,3,1,13697.0,1407338.0,Portland,Maine,4108,992.0
82261.0,129000.0,2,1,100000.0,598202.0,Whitehall,New York,12887,1352.0
91618.0,264900.0,3,2,99999.0,1137811.0,Montville,Connecticut,6353,1485.0
67217.0,520000.0,0,0,49299.44,677424.0,Newark,New Jersey,7112,0.0
80298.0,399000.0,3,2,58100.0,1001569.0,Kearny,New Jersey,7032,0.0
67217.0,485000.0,4,3,100000.0,1081800.0,Scotch Plains,New Jersey,7076,0.0
